In [1]:
import numpy as np
import pandas as pd
import math

# Read data from DataSets
mean1_pos = pd.read_csv("data/DS2_c1_m1.txt", header=None)
mean2_pos = pd.read_csv("data/DS2_c1_m2.txt", header=None)
mean3_pos = pd.read_csv("data/DS2_c1_m3.txt", header=None)
mean1_neg = pd.read_csv("data/DS2_c2_m1.txt", header=None)
mean2_neg = pd.read_csv("data/DS2_c2_m2.txt", header=None)
mean3_neg = pd.read_csv("data/DS2_c2_m3.txt", header=None)
cov1 = pd.read_csv("data/DS2_Cov1.txt", header=None)
cov2 = pd.read_csv("data/DS2_Cov2.txt", header=None)
cov3 = pd.read_csv("data/DS2_Cov3.txt", header=None)
# Drop end column (not useful to us)
mean1_pos.drop([20], axis=1, inplace=True)
mean2_pos.drop([20], axis=1, inplace=True)
mean3_pos.drop([20], axis=1, inplace=True)
mean1_neg.drop([20], axis=1, inplace=True)
mean2_neg.drop([20], axis=1, inplace=True)
mean3_neg.drop([20], axis=1, inplace=True)
cov1.drop([20], axis=1, inplace=True)
cov2.drop([20], axis=1, inplace=True)
cov3.drop([20], axis=1, inplace=True)
# Change to matrices
mean1_pos_m = mean1_pos.as_matrix()[0]
mean2_pos_m = mean2_pos.as_matrix()[0]
mean3_pos_m = mean3_pos.as_matrix()[0]
mean1_neg_m = mean1_neg.as_matrix()[0]
mean2_neg_m = mean2_neg.as_matrix()[0]
mean3_neg_m = mean3_neg.as_matrix()[0]
cov1_m = cov1.as_matrix()[0]
cov2_m = cov2.as_matrix()[0]
cov3_m = cov3.as_matrix()[0]

# Generate 2000 examples for each class
dataEx = 2000
class1_neg = pd.DataFrame(np.random.multivariate_normal(mean1_neg_m, cov1, dataEx))
class2_neg = pd.DataFrame(np.random.multivariate_normal(mean2_neg_m, cov2, dataEx))
class3_neg = pd.DataFrame(np.random.multivariate_normal(mean3_neg_m, cov3, dataEx))
class1_pos = pd.DataFrame(np.random.multivariate_normal(mean1_pos_m, cov1, dataEx))
class2_pos = pd.DataFrame(np.random.multivariate_normal(mean2_pos_m, cov2, dataEx))
class3_pos = pd.DataFrame(np.random.multivariate_normal(mean3_pos_m, cov3, dataEx))
# Add classification column and convert to matrix
class1_neg[20] = 0
class2_neg[20] = 0
class3_neg[20] = 0
class1_pos[20] = 1
class2_pos[20] = 1
class3_pos[20] = 1
class1_neg_m = class1_neg.as_matrix()
class2_neg_m = class2_neg.as_matrix()
class3_neg_m = class3_neg.as_matrix()
class1_pos_m = class1_pos.as_matrix()
class2_pos_m = class2_pos.as_matrix()
class3_pos_m = class3_pos.as_matrix()

# Split data into train, valid, test sets
allData1 = np.concatenate((class1_neg_m, class1_pos_m), axis=0)
allData2 = np.concatenate((class2_neg_m, class2_pos_m), axis=0)
allData3 = np.concatenate((class3_neg_m, class3_pos_m), axis=0)
np.random.shuffle(allData1)
np.random.shuffle(allData2)
np.random.shuffle(allData3)
trainSet1 = pd.DataFrame(allData1[0:2400])
trainSet2 = pd.DataFrame(allData2[0:2400])
trainSet3 = pd.DataFrame(allData3[0:2400])
validSet1 = pd.DataFrame(allData1[2400:3200])
validSet2 = pd.DataFrame(allData2[2400:3200])
validSet3 = pd.DataFrame(allData3[2400:3200])
testSet1 = pd.DataFrame(allData1[3200:4000])
testSet2 = pd.DataFrame(allData2[3200:4000])
testSet3 = pd.DataFrame(allData3[3200:4000])
# Concatenate data from the 3 Gaussian datasets
allData = np.concatenate((pd.DataFrame(allData1), pd.DataFrame(allData2), pd.DataFrame(allData3)), axis=0)
trainSet = pd.DataFrame(np.concatenate((trainSet1, trainSet2, trainSet3), axis=0))
validSet = pd.DataFrame(np.concatenate((validSet1, validSet2, validSet3), axis=0))
testSet = pd.DataFrame(np.concatenate((testSet1, testSet2, testSet3), axis=0))

### 4. Save completed datasets ###
pd.DataFrame(allData).to_csv("dataGenerated/DS2_data.csv", index=False, header=False)
trainSet.to_csv("dataGenerated/DS2_train.csv", index=False, header=False)
validSet.to_csv("dataGenerated/DS2_valid.csv", index=False, header=False)
testSet.to_csv("dataGenerated/DS2_test.csv", index=False, header=False)

In [3]:
### 5.1 GDA Approach ###

# Split training data
trainSet0 = trainSet[trainSet[20] == 0]
trainSet1 = trainSet[trainSet[20] == 1]
testOut = testSet[20]
# Drop end column
trainSet0.drop([20], axis=1, inplace=True)
trainSet1.drop([20], axis=1, inplace=True)
trainSet.drop([20], axis=1, inplace=True)
testSet.drop([20], axis=1, inplace=True)

# Get length of each training set
numData0 = len(trainSet0)
numData1 = len(trainSet1)
# Create nd.array versions of data sets
trainSet_arr = trainSet.as_matrix()
testSet_arr = testSet.as_matrix()

# Get probability of each training set
allDataPts = numData0 + numData1
prob0 = numData0 / allDataPts
prob1 = numData1 / allDataPts

# Get mean of each column
mean0 = np.array(trainSet0.mean())
mean1 = np.array(trainSet1.mean())

# Compute covariance matrices to get w0 & w1
cov0 = np.array(trainSet0-mean0)
cov1 = np.array(trainSet1-mean1)
s0 = np.matmul(cov0.T, cov0)
s1 = np.matmul(cov1.T, cov1)
cov = (s0 + s1) / allDataPts
cov_inv = np.linalg.inv(cov)
w0 = (math.log(prob0)-math.log(prob1)) - (np.matmul(np.matmul(mean0.T, cov_inv), mean0) - np.matmul(np.matmul(mean1.T, cov_inv), mean1))/2
w1 = np.matmul(cov_inv, (mean0 - mean1))
print("Covariance matrix: ", cov, "\n")
print("w0: ", w0)
print("w1: ", w1, "\n")

# Compute GDA model
outPredict = np.matmul(testSet, w1.T) + w0
sigmoid = np.exp(outPredict) / (np.exp(outPredict) + 1)
for i in range(len(sigmoid)):
    if (sigmoid[i] > 0.5):
        sigmoid[i] = 0
    else:
        sigmoid[i] = 1
        
# Compute confusion matrix values
tp, tn, fp, fn = 0, 0, 0, 0
for j in range(len(testOut)):
    predVal = sigmoid[j]
    actualVal = testOut[j]

    # Compare actual to predicted values
    if (predVal == 1 and predVal == actualVal):
        tp += 1
    elif (predVal == 1 and predVal != actualVal):
        fp += 1
    elif (predVal == 0 and predVal == actualVal):
        tn += 1
    elif (predVal == 0 and predVal != actualVal):
        fn += 1
        
# Create confusion matrix
confMatrix = [[0, 0], [0, 0]]
confMatrix[0][0] = tp
confMatrix[0][1] = fp
confMatrix[1][0] = fn
confMatrix[1][1] = tn
        
# Compute results
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f_measure = (2 * (precision * recall)) / (precision + recall)

# Display results
print("Confusion Matrix: ", confMatrix, "\n")
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F-measure: ", f_measure)

Covariance matrix:  [[ 7.56169367  5.64754268  5.03943775  5.13906646  4.55223492  6.07501614
   5.61651127  5.47711743  4.78996973  5.50180405  5.43352109  5.2442264
   5.48374432  6.17417384  5.48216603  5.84502266  5.612228    5.29520831
   5.34138188  5.85475509]
 [ 5.64754268  7.70816138  5.53592544  5.50191941  5.44724338  6.7257495
   6.40898454  5.76894248  5.08351081  5.65267822  5.27305654  5.69888356
   5.48083503  6.39271987  5.98635782  6.30688827  5.98772765  5.34417968
   6.13117252  6.20283434]
 [ 5.03943775  5.53592544  6.99794496  5.2872627   5.15774254  5.65569679
   5.92321446  4.88120725  5.03845943  5.27570563  5.01667934  5.29912099
   5.51262949  6.21791229  5.33952636  5.27536418  5.70613317  4.70000908
   5.68174748  6.03514885]
 [ 5.13906646  5.50191941  5.2872627   6.60019364  4.4256281   5.85194092
   5.742914    5.54676071  4.92313805  5.59054672  4.96471657  5.28627163
   5.1257205   5.52809292  5.46387196  5.40322586  5.42758112  5.17622819
   5.64739102

In [4]:
### 5.2 k-NN Approach ###

# Store optimal values
k_opt = 0
confMatrix_opt = [[0, 0], [0, 0]]
accuracy_opt = 0.0

# Compute k-NN up to k=30
for k in range(1, 31):
    tp, tn, fp, fn = 0, 0, 0, 0
    for i in range(len(testOut)):
        x = trainSet_arr
        y_i = testSet_arr[i]
        
        # Compute and sort distances
        distances = np.power(np.linalg.norm(x - y_i, axis=1), 2)
        neighbours = distances.argsort()[:k]
        if (trainSet_arr[neighbours, 19].sum() > 0):
            predVal = 1
        else:
            predVal = 0
        actualVal = testOut[i]
        
        # Compare actual to predicted values
        if (predVal == 1 and predVal == actualVal):
            tp += 1
        elif (predVal == 1 and predVal != actualVal):
            fp += 1
        elif (predVal == 0 and predVal == actualVal):
            tn += 1
        elif (predVal == 0 and predVal != actualVal):
            fn += 1
        
    # Compute and display accuracy
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    print("k: {}, Accuracy: {}".format(k, accuracy))
    
    # Update highest accuracy and store corresponding confusion matrix
    if (accuracy > accuracy_opt):
        accuracy_opt = accuracy
        k_opt = k
        confMatrix_opt[0][0] = tp
        confMatrix_opt[0][1] = fp
        confMatrix_opt[1][0] = fn
        confMatrix_opt[1][1] = tn
        
# Compute results from optimal values
tp = confMatrix_opt[0][0]
fp = confMatrix_opt[0][1]
fn = confMatrix_opt[1][0]
tn = confMatrix_opt[1][1]
precision_opt = tp / (tp + fp)
recall_opt = tp / (tp + fn)
f_measure_opt = (2 * (precision_opt * recall_opt)) / (precision_opt + recall_opt)

# Display results
print("\nOptimal k value: ", k_opt)
print("Confusion Matrix: ", confMatrix_opt, "\n")
print("Accuracy: ", accuracy_opt)
print("Precision: ", precision_opt)
print("Recall: ", recall_opt)
print("F-measure: ", f_measure_opt)

k: 1, Accuracy: 0.47041666666666665
k: 2, Accuracy: 0.465
k: 3, Accuracy: 0.4683333333333333
k: 4, Accuracy: 0.46875
k: 5, Accuracy: 0.4708333333333333
k: 6, Accuracy: 0.46958333333333335
k: 7, Accuracy: 0.4708333333333333
k: 8, Accuracy: 0.47291666666666665
k: 9, Accuracy: 0.47541666666666665
k: 10, Accuracy: 0.47
k: 11, Accuracy: 0.4691666666666667
k: 12, Accuracy: 0.47041666666666665
k: 13, Accuracy: 0.4716666666666667
k: 14, Accuracy: 0.46875
k: 15, Accuracy: 0.47
k: 16, Accuracy: 0.47208333333333335
k: 17, Accuracy: 0.46875
k: 18, Accuracy: 0.47041666666666665
k: 19, Accuracy: 0.47041666666666665
k: 20, Accuracy: 0.4683333333333333
k: 21, Accuracy: 0.47041666666666665
k: 22, Accuracy: 0.4683333333333333
k: 23, Accuracy: 0.46875
k: 24, Accuracy: 0.46625
k: 25, Accuracy: 0.46708333333333335
k: 26, Accuracy: 0.46541666666666665
k: 27, Accuracy: 0.4666666666666667
k: 28, Accuracy: 0.46541666666666665
k: 29, Accuracy: 0.465
k: 30, Accuracy: 0.46791666666666665

Optimal k value:  9
Conf